# Extended overview figures

This notebook creates the extended overview plots from the SI information of the publication.

In [1]:
import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import os

sys.path.append("../../")
sys.path.append("../../covid19_inference")
sys.path.append("../")

import covid19_soccer
from covid19_soccer.plot.utils import get_from_trace
#import covid19_inference as cov19
from header_plotting import *

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
def load(fstr):
    with open(fstr, "rb") as f:
         return pickle.load(f)

countries = ['England', 'Czechia', 'Scotland', 'Spain', 'Germany', 'Austria',
       'France', 'Slovakia', 'Belgium', 'Italy', 'Portugal',
       'Netherlands', 'GB']
traces, models, dls = [], [], []
for country in tqdm(countries):
    model = None
    fstr=lambda tune, draws, max_treedepth: (f"/data.nst/smohr/covid19_soccer_data/main_traces/run"+
        f"-beta=False"+
        f"-country={country}"+
        f"-offset_data=0"+
        f"-prior_delay=-1"+
        f"-median_width_delay=1.0"+
        f"-interval_cps=10.0"+
        f"-f_fem=0.33"+
        f"-len=normal"+
        f"-abs_sine=False"+
        f"-t={tune}"+
        f"-d={draws}"+
        f"-max_treedepth={max_treedepth}.pkl")
    if country == "Spain":
        fstr=lambda tune, draws, max_treedepth: (f"/data.nst/share/soccer_project/covid_uefa_traces14_spain/run"+
        f"-beta=False"+
        f"-country={country}"+
        f"-offset_data=0"+
        f"-prior_delay=-1"+
        f"-median_width_delay=1.0"+
        f"-interval_cps=10.0"+
        f"-f_fem=0.33"+
        f"-len=normal"+
        f"-abs_sine=False"+
        f"-t={tune}"+
        f"-d={draws}"+
        f"-max_treedepth={max_treedepth}.pkl")   
    if country == "GB":
        fstr=lambda tune, draws, max_treedepth: (f"/data.nst/share/soccer_project/covid_uefa_traces15/run"+
        f"-beta=False"+
        f"-country={country}"+
        f"-offset_data=0"+
        f"-prior_delay=-1"+
        f"-median_width_delay=1.0"+
        f"-interval_cps=10.0"+
        f"-f_fem=0.33"+
        f"-len=normal"+
        f"-abs_sine=False"+
        f"-t={tune}"+
        f"-d={draws}"+
        f"-max_treedepth={max_treedepth}.pkl")   
    if os.path.exists(fstr(4000, 8000, 12)):
        try:
            model, trace = load(fstr(4000, 8000, 12))
            print(f"Use 8000 sample runs for {country}")
        except:
            pass
    if model is None and os.path.exists(fstr(2000, 4000, 12)):
        try:
            model, trace = load(fstr(2000, 4000, 12))
            print(f"Use 4000 sample runs for {country}")
        except:
            pass
    if model is None and os.path.exists(fstr(1000, 1500, 12)):
        try: 
            model, trace = load(fstr(1000, 1500, 12))
            print(f"Use 1500 sample runs for {country}")
        except:
            pass
    if model is None and os.path.exists(fstr(1000, 2000, 12)):
        try: 
            model, trace = load(fstr(1000, 2000, 12))
            print(f"Use 2000 sample runs for {country}")
        except:
            pass
    if model is None and os.path.exists(fstr(500, 1000, 12)):
        try: 
            model, trace = load(fstr(500, 1000, 12))
            print(f"Use 1000 sample runs for {country}")
        except:
            pass
    if model is None: 
        print(fstr(900, 800, 99), " not found")
        continue
    
    # Remove chains with likelihood larger than -200, should only be the case for 2 chains in France
    mask = (np.mean(trace.sample_stats.lp, axis=1)>-200)
    trace.posterior = trace.posterior.sel(chain=~mask)
    
    dl = covid19_soccer.dataloader.Dataloader_gender(countries=[country])
    models.append(model)
    traces.append(trace)
    dls.append(dl)

  0%|          | 0/13 [00:00<?, ?it/s]

Use 4000 sample runs for England
Use 4000 sample runs for Czechia


In [ ]:
from covid19_soccer.plot.overview import single_extended,single_extended_v2

"""
country2ylim_inbalance = {
    "England": [-0.15,0.3],
    "Scotland": [-0.15,0.45],
    "Slovakia": [-0.6,0.6],
    "Italy": [-0.3,0.45],
    "Czechia": [-0.3,0.45]
}
"""
ylim_Imbalance = [-0.19,0.25]
ylim_Rnoise = [0.9,1.2]
ylim_Rbase = [0.5,2]

for trace,model,dl in zip(traces,models,dls):

    country = dl.countries[0]
    
    if country == "Netherlands":
        ylim_rbase = [-0.5,5]
    else:
        ylim_rbase = ylim_Rbase
    
    if country == "Scotland":
        ylim_imbalance = [-0.19,0.46]
    else:
        ylim_imbalance = ylim_Imbalance
        
    if country == "Italy":
        ylim_incidence=[0,220]
    else:
        ylim_incidence=None
        
    fig0 = single_extended_v2(
        trace,
        model,
        dl,
        ylim_imbalance=ylim_imbalance,
        ylim_incidence=ylim_incidence,
        ylim_rnoise=ylim_Rnoise,
        ylim_rbase=ylim_rbase
    )
    if country == "Czechia":
        country = "Czech Republic"
    if country == "United Kingdom":
        country = "GB (England + Scotland)"
    fig0.suptitle(f"{country}")

    fig0.savefig(
        f"../figures/SI/extended_overview/extended_overview_{dl.countries[0].replace(' ', '_')}.pdf",
        transparent=True,
        dpi=300,
        bbox_inches='tight'
    )
    fig0.savefig(
        f"../figures/SI/extended_overview/extended_overview_{dl.countries[0].replace(' ', '_')}.png",
        transparent=True,
        dpi=300,
        bbox_inches='tight'
    )
    plt.show()
    plt.close(fig=fig0)

In [ ]:
from covid19_soccer.plot.other import soccer_related_cases_overview

fig,ax = plt.subplots(1,1,figsize=(4,3))

soccer_related_cases_overview(
        ax,
        traces,
        models,
        dls,
        plot_flags=True,
        ypos_flags=-20,
        remove_outliers=True,
        bw=0.1,
        country_order=None,
        overall_effect_trace=None,
        vertical=True,
)
plt.show()

In [ ]:
import arviz as az

In [ ]:
rhats=[]
for tr in traces:
    rhats.append(az.rhat(tr))

In [ ]:
for i,(rhat,country) in enumerate(zip(rhats,countries)):
    chosen_vars = []
    for var in list(rhat.data_vars):
        if "factor_female" in var:
            chosen_vars.append(var)
        if "alpha" in var:
            chosen_vars.append(var)

    rhats_R_t = []
    rhat_max = rhat.max()
    for var in chosen_vars:
        rhats_R_t.append(float(rhat_max[var]))
    rhats_all = []
    for var in rhat.data_vars:
        rhats_all.append(float(rhat_max[var]))
    print(f"{i} {country}: ({len(traces[i].posterior.chain)}, {len(traces[i].posterior.draw)})")
    print(f"\tmax rhat of R_t {max(rhats_R_t)} ")
    print(f"\tmax overall rhat {max(rhats_all)} ")